In [1]:
import os
import json
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import numpy as np

# Use a clean theme for the interactive plot
pio.templates.default = "plotly_white"
pio.renderers.default = "notebook"

# --- Configuration ---
RESULTS_DIR = 'client_results'

# Get subdirectories
subdirs = [d for d in os.listdir(RESULTS_DIR) if os.path.isdir(os.path.join(RESULTS_DIR, d))]

if len(subdirs) != 2:
    print(f"Error: Expected 2 subdirectories in {RESULTS_DIR}, found {len(subdirs)}")
    print(f"Subdirectories found: {subdirs}")
else:
    print(f"Found 2 subdirectories: {subdirs[0]} and {subdirs[1]}")

# Professional color palette
combined_color = '#F18F01'  # Vibrant orange for combined baseline
tool_color = '#2E4057'      # Deep blue-grey for all tools

# Helper function to safely get nested dictionary values
def safe_get(data, *keys, default=0):
    """Safely navigate nested dictionaries with a default value."""
    result = data
    for key in keys:
        if isinstance(result, dict):
            result = result.get(key, {})
        else:
            return default
    return result if result != {} else default

# Enhanced log scale tick function
def get_log_ticks(data):
    """Generate appropriate logarithmic scale ticks."""
    if len(data) == 0:
        return [0.1, 1, 10, 100, 1000], ['0.1', '1', '10', '100', '1000'], []
    
    filtered_data = [x for x in data if x > 0 and np.isfinite(x)]
    if not filtered_data:
        return [0.1, 1, 10, 100, 1000], ['0.1', '1', '10', '100', '1000'], []
    
    min_val = min(filtered_data)
    max_val = max(filtered_data)
    
    try:
        min_pow = int(np.floor(np.log10(min_val)))
        max_pow = int(np.ceil(np.log10(max_val)))
        
        if max_pow - min_pow > 10:
            max_pow = min_pow + 10
        
        major_ticks = [10**i for i in range(min_pow, max_pow + 1)]
        tick_labels = [f'{10**i:.0e}' if i < -2 or i > 3 else str(10**i) for i in range(min_pow, max_pow + 1)]
        
        minor_ticks = []
        for i in range(min_pow, max_pow + 1):
            base = 10 ** i
            minor_ticks.extend([base * j for j in range(2, 10)])
        
        minor_ticks = [t for t in minor_ticks if min_val <= t <= max_val * 2]
        
        return major_ticks, tick_labels, minor_ticks
    except (OverflowError, ValueError):
        return [0.1, 1, 10, 100, 1000], ['0.1', '1', '10', '100', '1000'], []

# Function to process a directory
def process_directory(dir_path):
    tool_data = {}
    
    for filename in os.listdir(dir_path):
        if not filename.endswith('.json'):
            continue
        
        filepath = os.path.join(dir_path, filename)
        
        try:
            with open(filepath, 'r') as f:
                data = json.load(f)
            
            if 'tool_name' in data and 'tool' not in data:
                tool_name = data.get('tool_name')
                if tool_name not in tool_data:
                    tool_data[tool_name] = {
                        'type': 'Tool',
                        'abc_latencies': [],
                        'raw_latencies': [],
                        'ab_tcp': [],
                        'bc_tcp': [],
                        'ab_icmp': [],
                        'bc_icmp': [],
                    }
                
                measurements = data.get('measurements', {})
                bc_tcp_one_way = safe_get(measurements, 'tcp', 'one_way_average_ms', default=0)
                if bc_tcp_one_way > 0:
                    tool_data[tool_name]['bc_tcp'].append(bc_tcp_one_way)
                
                bc_icmp_one_way = safe_get(measurements, 'icmp', 'one_way_average_ms', default=0)
                if bc_icmp_one_way > 0:
                    tool_data[tool_name]['bc_icmp'].append(bc_icmp_one_way)
                
                continue
            
            tool_name = data.get('tool')
            if tool_name is None:
                continue
            
            is_combined = 'combined' in tool_name.lower()
            is_baseline = 'combined' not in tool_name.lower() and any(x in tool_name.lower() for x in ['baseline', 'direct'])
            
            if tool_name not in tool_data:
                if is_combined:
                    data_type = 'combined'
                elif is_baseline:
                    data_type = 'baseline'
                else:
                    data_type = 'Tool'
                
                tool_data[tool_name] = {
                    'type': data_type,
                    'abc_latencies': [],
                    'raw_latencies': [],
                    'ab_tcp': [],
                    'bc_tcp': [],
                    'ab_icmp': [],
                    'bc_icmp': [],
                }
            
            runs = data.get('original_runs', [data])
            
            for run in runs:
                avg_socket_based = safe_get(run, 'averageSocketBased', default=0) / 1000
                if avg_socket_based > 0:
                    tool_data[tool_name]['abc_latencies'].append(avg_socket_based)
                
                measurements = run.get('socketBasedMeasurements', [])
                for meas in measurements:
                    latency = meas.get('oneWayLatency', 0) / 1000
                    if latency > 0:
                        tool_data[tool_name]['raw_latencies'].append(latency)
                
                ab_pings = run.get('abPings', {})
                ab_tcp_one_way = safe_get(ab_pings, 'tcp', 'oneWayAverage', default=0)
                if ab_tcp_one_way > 0:
                    tool_data[tool_name]['ab_tcp'].append(ab_tcp_one_way)
                
                ab_icmp_one_way = safe_get(ab_pings, 'icmp', 'oneWayAverage', default=0)
                if ab_icmp_one_way > 0:
                    tool_data[tool_name]['ab_icmp'].append(ab_tcp_one_way)
                
                bc_results = run.get('bcResults', {})
                measurements = bc_results.get('measurements', {})
                bc_tcp_one_way = safe_get(measurements, 'tcp', 'one_way_average_ms', default=0)
                if bc_tcp_one_way > 0:
                    tool_data[tool_name]['bc_tcp'].append(bc_tcp_one_way)
                
                bc_icmp_one_way = safe_get(measurements, 'icmp', 'one_way_average_ms', default=0)
                if bc_icmp_one_way > 0:
                    tool_data[tool_name]['bc_icmp'].append(bc_icmp_one_way)
        
        except Exception as e:
            print(f"⚠️ Could not load or parse {filename}. Error: {e}")
            continue
    
    # Convert to list for DataFrame
    final_data_list = []
    for tool_name, data_dict in tool_data.items():
        final_data_list.append({
            'Tool': tool_name,
            'Type': data_dict['type'],
            'ABC_Latency': np.mean(data_dict['abc_latencies']) if data_dict['abc_latencies'] else 0,
            'ABC_Std': np.std(data_dict['abc_latencies']) if len(data_dict['abc_latencies']) > 1 else 0,
            'AB_ICMP': np.mean(data_dict['ab_icmp']) if data_dict['ab_icmp'] else 0,
            'BC_ICMP': np.mean(data_dict['bc_icmp']) if data_dict['bc_icmp'] else 0,
            'AB_TCP': np.mean(data_dict['ab_tcp']) if data_dict['ab_tcp'] else 0,
            'BC_TCP': np.mean(data_dict['bc_tcp']) if data_dict['bc_tcp'] else 0,
            'raw_latencies': data_dict['raw_latencies'],
        })
    
    # Create DataFrame, sort by type then tool name
    type_order = {'combined': 0, 'baseline': 1, 'Tool': 2}
    df = pd.DataFrame(final_data_list)
    df['type_order'] = df['Type'].map(type_order)
    df = df.sort_values(by=['type_order', 'Tool'], ascending=[True, True]).reset_index(drop=True)
    df = df.drop('type_order', axis=1)
    
    return df

# Process both directories
dfs = {}
for subdir in subdirs:
    dir_path = os.path.join(RESULTS_DIR, subdir)
    print(f"\n📂 Processing directory: {subdir}")
    dfs[subdir] = process_directory(dir_path)
    print(f"✅ Processed {len(dfs[subdir])} tools from {subdir}")

# Get consistent x-axis labels (union of all tools from both directories)
all_tools = set()
for df in dfs.values():
    all_tools.update(df['Tool'].tolist())
all_tools = sorted(list(all_tools))

print(f"\n📊 Creating visualizations with {len(all_tools)} unique tools")

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# --- Create Stacked Bar Graphs ---
from plotly.subplots import make_subplots
fig_bar = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.02,
    row_heights=[0.5, 0.5]
)

for idx, (subdir, df) in enumerate(dfs.items()):
    row_num = idx + 1
    
    for i, row in df.iterrows():
        # Determine color based on type
        if row['Type'] == 'combined':
            color = combined_color
            border_width = 0  # Thinner border for combined baseline
        else:
            color = tool_color
            border_width = 4  # Original thick border for tunneling tools
        
        fig_bar.add_trace(
            go.Bar(
                x=[row['Tool']],
                y=[row['ABC_Latency']],
                name=row['Tool'],
                marker=dict(
                    color=color,
                    line=dict(color='black', width=border_width),
                    opacity=0.85
                ),
                error_y=dict(
                    type='data',
                    array=[row['ABC_Std']],
                    visible=True,
                    color='rgba(50, 50, 50, 0.8)',
                    thickness=2.5,
                    width=8
                ),
                hovertemplate=(
                    f"<b>{row['Tool']}</b><br>"
                    f"Mean: %{{y:.2f}} ms<br>"
                    f"Std: {row['ABC_Std']:.2f} ms<br>"
                    "<extra></extra>"
                ),
                showlegend=False
            ),
            row=row_num, col=1
        )

# Update x-axis for bottom subplot only
fig_bar.update_xaxes(
#     title_text='<b>Network Configuration</b>',
    title_font=dict(family='Arial, sans-serif', size=24, color='black', weight='bold'),
    tickmode='array',
    tickvals=all_tools,
    ticktext=[f'<b>{label}</b>' for label in all_tools],
    tickfont=dict(family='Arial, sans-serif', size=30, color='black', weight='bold'),
    showline=True,
    linewidth=3,
    linecolor='black',
    tickangle=25,
    showgrid=False,
    mirror=True,
    row=2, col=1
)

# Update x-axis for top subplot (mirror for box)
fig_bar.update_xaxes(
    showline=True,
    linewidth=2,
    linecolor='black',
    mirror=True,
    row=1, col=1
)

# Y-axis for row 1
fig_bar.update_yaxes(
    title_text='<b>Average Latency (ms)</b>',
    row=1, col=1,
    mirror=True,
    title_font=dict(family='Arial, sans-serif', size=34, color='black', weight='bold'),
    showline=True, 
    linewidth=3, 
    linecolor='black',
    tickfont=dict(family='Arial, sans-serif', size=30, color='black', weight='bold'),
    showgrid=False
)

# Y-axis for row 2
fig_bar.update_yaxes(
    title_text='<b>Average Latency (ms)</b>',
    row=2, col=1,
    mirror=True,
    title_font=dict(family='Arial, sans-serif', size=34, color='black', weight='bold'),
    showline=True, 
    linewidth=3, 
    linecolor='black',
    tickfont=dict(family='Arial, sans-serif', size=30, color='black', weight='bold'),
    showgrid=False
)

fig_bar.update_layout(
    title=dict(
#         text='<b>End-to-End Latency Performance: Baseline vs. Tunneling Tools</b>',
        x=0.5,
        xanchor='center',
        font=dict(family='Arial, sans-serif', size=24, color='black', weight='bold')
    ),
    plot_bgcolor='white',
    paper_bgcolor='white',
    height=1200,
    width=1400,
    margin=dict(l=100, r=100, t=120, b=150),
    showlegend=False,
    annotations=[
        # SF-SF label for top subplot
        dict(
            text='<b>SF-SF</b>',
            xref='x', yref='y',
            x=10.5, y=190,
            xanchor='right', yanchor='top',
            showarrow=False,
            font=dict(family='Arial, sans-serif', size=30, color='black'),
            bgcolor='rgba(255, 255, 255, 0.8)',
            bordercolor='black',
            borderwidth=2,
            borderpad=8
        ),
        # SGP-AMS label for bottom subplot
        dict(
            text='<b>SGP-AMS</b>',
            xref='x2', yref='y2',
            x=10.5, y=255,
            xanchor='right', yanchor='top',
            showarrow=False,
            font=dict(family='Arial, sans-serif', size=30, color='black'),
            bgcolor='rgba(255, 255, 255, 0.8)',
            bordercolor='black',
            borderwidth=2,
            borderpad=8
        )
    ]
)

fig_bar.show()

In [ ]:
# --- Create Stacked Box Plots ---
fig_box = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.02,
    row_heights=[0.5, 0.5]
)

for idx, (subdir, df) in enumerate(dfs.items()):
    row_num = idx + 1
    
    # Get all raw latencies for proper tick calculation
    all_latencies = []
    for _, row in df.iterrows():
        all_latencies.extend(row['raw_latencies'])
    
    major_ticks, tick_labels, minor_ticks = get_log_ticks(all_latencies)
    
    for i, row in df.iterrows():
        # Determine color based on type
        if row['Type'] == 'combined':
            color = combined_color
        else:
            color = tool_color
        
        fig_box.add_trace(
            go.Box(
                y=row['raw_latencies'],
                name=row['Tool'],
                marker=dict(
                    color=color,
                    size=8,
                    opacity=0.6,
                    line=dict(width=1, color='black')
                ),
                line=dict(color='black', width=4),
                fillcolor=color,
                opacity=0.7,
                width=0.8,
                showlegend=False,
                boxpoints='outliers',
                hovertemplate=(
                    f"<b>{row['Tool']}</b><br>"
                    "Value: %{y:.2f} ms<br>"
                    "<extra></extra>"
                ),
                x=[row['Tool']] * len(row['raw_latencies'])
            ),
            row=row_num, col=1
        )
    
    # Update y-axes with log scale for each subplot
    fig_box.update_yaxes(
        title_text='<b>One-Way Latency (ms)</b>',
        title_font=dict(family='Arial, sans-serif', size=24, color='black'),
        type='log',
        tickvals=major_ticks,
        ticktext=tick_labels,
        minor=dict(
            tickvals=minor_ticks,
            ticklen=4,
            tickwidth=0.8,
            tickcolor='rgba(0, 0, 0, 0.3)',
            showgrid=False
        ),
        tickfont=dict(family='Arial, sans-serif', size=24, color='black'),
        showline=True,
        linewidth=4,
        mirror=True,
        linecolor='black',
        showgrid=False,
        row=row_num, col=1
    )

# Update x-axis for bottom subplot
fig_box.update_xaxes(
    title_font=dict(family='Arial, sans-serif', size=24, color='black'),
    tickmode='array',
    tickvals=all_tools,
    ticktext=[f'<b>{label}</b>' for label in all_tools],
    tickfont=dict(family='Arial, sans-serif', size=24, color='black'),
    showline=True,
    linewidth=4,
    mirror=True,
    linecolor='black',
    tickangle=45,
    showgrid=False,
    row=2, col=1
)

# Update x-axis for top subplot (mirror for box)
fig_box.update_xaxes(
    showline=True,
    linewidth=4,
    linecolor='black',
    mirror=True,
    row=1, col=1
)

fig_box.update_layout(
    title=dict(
        text='<b>Distribution of One-Way Latency Measurements</b>',
        x=0.5,
        xanchor='center',
        font=dict(family='Arial, sans-serif', size=24, color='black')
    ),
    plot_bgcolor='white',
    paper_bgcolor='white',
    height=1200,
    width=1400,
    margin=dict(l=100, r=100, t=120, b=150),
    showlegend=False,
    boxmode='group',
    annotations=[
        # SF-SF label for top subplot
        dict(
            text='<b>SF-SF</b>',
            xref='x', yref='paper',
            x=10.5, y=0.97,
            xanchor='right', yanchor='top',
            showarrow=False,
            font=dict(family='Arial, sans-serif', size=20, color='black'),
            bgcolor='rgba(255, 255, 255, 0.8)',
            bordercolor='black',
            borderwidth=2,
            borderpad=8
        ),
        # SGP-AMS label for bottom subplot
        dict(
            text='<b>SGP-AMS</b>',
            xref='x2', yref='paper',
            x=10.5, y=0.47,
            xanchor='right', yanchor='top',
            showarrow=False,
            font=dict(family='Arial, sans-serif', size=20, color='black'),
            bgcolor='rgba(255, 255, 255, 0.8)',
            bordercolor='black',
            borderwidth=2,
            borderpad=8
        )
    ]
)

fig_box.show()